In [143]:
import os
import requests
import json
import pandas as pd

In [144]:
aprx = arcpy.mp.ArcGISProject("current")
map = aprx.listMaps()[0]
layers = map.listLayers()
print (layers[0].name)

Topographic


In [145]:
wrldLayerPath = r"C:\WorkSpace\adm1_polygons.shp"
geoLayer = map.addDataFromPath(wrldLayerPath)


In [146]:
#access the COD Services API list of COD locations and themes
COD_External_URL = 'https://beta.itos.uga.edu/CODV2API/api/v1/themes'
response = requests.get(COD_External_URL)
dataD = json.loads(json.dumps(response.json()))
#review contents and check to see where there may be relavency with the world layer
for i in range (0, len(dataD)):
    print(dataD[i])

{'theme_name': 'Administrative Boundary', 'comment': '', 'countries': ['abw', 'afg', 'ago', 'aia', 'alb', 'ant', 'are', 'arg', 'arm', 'atg', 'aze', 'bdi', 'bel', 'ben', 'bfa', 'bgd', 'bgr', 'bhr', 'bhs', 'blm', 'blr', 'blz', 'bmu', 'bol', 'bra', 'brb', 'btn', 'bwa', 'caf', 'chl', 'chn', 'cmr', 'cod', 'cog', 'col', 'com', 'cpv', 'cri', 'cub', 'cuw', 'cym', 'dji', 'dma', 'dom', 'dza', 'ecu', 'egy', 'eri', 'esh', 'eth', 'fji', 'fsm', 'gab', 'geo', 'gha', 'gin', 'glp', 'gmb', 'gnb', 'gnq', 'grc', 'grd', 'gtm', 'guf', 'guy', 'hnd', 'hti', 'hun', 'idn', 'irn', 'irq', 'jam', 'jpn', 'kaz', 'kdn', 'ken', 'kgz', 'khm', 'kir', 'kna', 'kwt', 'lbn', 'lbr', 'lby', 'lca', 'lka', 'lso', 'maf', 'mar', 'mda', 'mdg', 'mdv', 'mex', 'mhl', 'mkd', 'mli', 'mng', 'moz', 'mrt', 'msr', 'mtq', 'mus', 'mwi', 'mys', 'nam', 'ner', 'nga', 'nic', 'npl', 'omn', 'pak', 'pan', 'per', 'phl', 'png', 'pol', 'pri', 'pry', 'pse', 'qat', 'rou', 'rus', 'rwa', 'sau', 'sdn', 'sen', 'slb', 'sle', 'slv', 'som', 'ssd', 'stp', 'sur'

In [147]:
#arrange output of the COD Services API to show the countries as rows

df= pd.json_normalize(dataD, record_path=['countries'], meta=['theme_name'])
update_array = df.to_records()
recs = update_array
dts = {'names': ('id', 'iso3', 'country'), 'formats':(numpy.uint8, 'U13', '<S256')}
array = numpy.rec.fromrecords(recs, dtype=dts)
df

,0,theme_name
0,abw,Administrative Boundary
1,afg,Administrative Boundary
2,ago,Administrative Boundary
3,aia,Administrative Boundary
4,alb,Administrative Boundary
...,...,...
290,vnm,Population Statistics
291,vut,Population Statistics
292,zaf,Population Statistics
293,zmb,Population Statistics


In [148]:
array['iso3'] = numpy.char.upper(array['iso3'])
df = pd.DataFrame(array)
df

,id,iso3,country
0,0,ABW,b'Administrative Boundary'
1,1,AFG,b'Administrative Boundary'
2,2,AGO,b'Administrative Boundary'
3,3,AIA,b'Administrative Boundary'
4,4,ALB,b'Administrative Boundary'
...,...,...,...
290,34,VNM,b'Population Statistics'
291,35,VUT,b'Population Statistics'
292,36,ZAF,b'Population Statistics'
293,37,ZMB,b'Population Statistics'


In [189]:
#Array to standalone table
output_table = "CODCountries14"
arcpy.da.NumPyArrayToTable(array, os.path.join(arcpy.env.workspace, output_table))
#Load the features with geometry and corresponding ISO 3 country values
inFeatures = geoLayer
inField = "adm0_src"
joinTable = os.path.join(arcpy.env.workspace, output_table)
joinField = "iso3" 
outFeature = "CODCoverageDemo.gdb/CountryCODS"
jt = arcpy.AddJoin_management(inFeatures, inField, joinTable, joinField)
print (jt)
arcpy.CopyFeatures_management(jt, outFeature)
aprx = arcpy.mp.ArcGISProject("current")
map = aprx.listMaps()[0]
layers = map.listLayers()


adm1_polygons


In [188]:
#Update layer to show countries with CODs 
newLayer = layers[0]
sym = newLayer.symbology
sym.updateRenderer("UniqueValueRenderer")
sym.valueField = "iso3"
r = sym.renderer
r.fields = "['iso3']"#['adm1_polygons.CODCountries10.iso3']
sym.renderer = r
sym.renderer.fields = "['iso3']" #['adm1_polygons.CODCountries10.iso3']
print (r.fields)
#sym.addAllValues()
#for grp in sym.renderer.groups:
#    for item in grp.items:
#        value = item.values[0][0]
#        if value == ' ':
#            item.symbol.color = {'RGB': [255, 12, 12, 100]}
newLayer.symbology = sym


['adm1_polygons_adm1_id']


In [178]:
print (sym.renderer.fields)
sym.renderer.fields[0] = "['adm1_polygons.CODCountries10.iso3']"
print (sym.renderer.fields)

['adm1_polygons.adm1_id']
['adm1_polygons.adm1_id']
